In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flu-shot-prediction/training_set_features.csv
/kaggle/input/flu-shot-prediction/test_set_features.csv
/kaggle/input/flu-shot-prediction/training_set_labels.csv
/kaggle/input/flu-shot-prediction/submission_format.csv


**Loading our dataset**

In [2]:
X = pd.read_csv("../input/flu-shot-prediction/training_set_features.csv")
test=pd.read_csv("../input/flu-shot-prediction/test_set_features.csv")

In [3]:
# created a target feature to differentiate between test and train data
X.loc[:,"target"]=1
test.loc[:,"target"]=-1

In [4]:
# merging the train and test data to label encode simultaneously
data=pd.concat([X, test])

In [5]:
X

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,target
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,1
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,1
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,1
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,1
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN,1
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea,1
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN,1
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,Married,Rent,Employed,lrircsnp,Non-MSA,1.0,0.0,fcxhlnwr,haliazsg,1


In [6]:
test

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,target
0,26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi,-1
1,26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp,-1
2,26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik,-1
3,26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN,-1
4,26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26703,53410,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,dqpwygqj,"MSA, Principle City",1.0,1.0,NaN,NaN,-1
26704,53411,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,Married,Rent,Employed,qufhixun,Non-MSA,1.0,3.0,fcxhlnwr,vlluhbov,-1
26705,53412,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Not Married,Rent,Not in Labor Force,qufhixun,"MSA, Not Principle City",1.0,0.0,NaN,NaN,-1
26706,53413,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,Married,Own,Not in Labor Force,bhuqouqj,"MSA, Not Principle City",1.0,0.0,NaN,NaN,-1


In [7]:
data

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,target
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,1
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,1
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,1
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,1
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26703,53410,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,dqpwygqj,"MSA, Principle City",1.0,1.0,NaN,NaN,-1
26704,53411,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,Married,Rent,Employed,qufhixun,Non-MSA,1.0,3.0,fcxhlnwr,vlluhbov,-1
26705,53412,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Not Married,Rent,Not in Labor Force,qufhixun,"MSA, Not Principle City",1.0,0.0,NaN,NaN,-1
26706,53413,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,Married,Own,Not in Labor Force,bhuqouqj,"MSA, Not Principle City",1.0,0.0,NaN,NaN,-1


In [8]:
# created a list called features containing all columns to be label encoded
features= [x for x in data.columns if x not in ["respondent_id","target"]]

In [9]:
len(features)

35

In [10]:
from sklearn import preprocessing

In [11]:
for feat in features:
    lbl_enc = preprocessing.LabelEncoder()
    temp_col = data[feat].fillna("None").astype(str)
    data.loc[:,feat] = lbl_enc.fit_transform(temp_col)

In [12]:
data

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,target
0,0,1,0,0,0,0,0,0,1,1,...,2,1,2,8,2,0,0,0,0,1
1,1,3,2,0,1,0,1,0,1,1,...,2,2,0,1,0,0,0,13,20,1
2,2,1,1,0,1,0,0,0,0,0,...,2,1,0,9,0,2,0,15,22,1
3,3,1,1,0,1,0,1,1,0,0,...,2,2,2,5,1,0,0,0,0,1
4,4,2,1,0,1,0,1,1,0,1,...,0,1,0,9,0,1,0,19,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26703,53410,1,1,0,1,0,1,0,0,1,...,1,0,1,2,1,1,1,0,0,-1
26704,53411,3,1,0,1,0,1,1,1,1,...,0,2,0,9,2,1,3,5,19,-1
26705,53412,0,1,0,0,0,0,0,0,0,...,2,2,2,9,0,1,0,0,0,-1
26706,53413,3,1,0,1,0,1,0,1,0,...,0,1,2,1,0,1,0,0,0,-1


We,have successfully filled all the missing values and then label encoded it

In [13]:
#Now we seperate our training and test data
X = data[data.target != -1]
test = data[data.target == -1]

In [14]:
X

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,target
0,0,1,0,0,0,0,0,0,1,1,...,2,1,2,8,2,0,0,0,0,1
1,1,3,2,0,1,0,1,0,1,1,...,2,2,0,1,0,0,0,13,20,1
2,2,1,1,0,1,0,0,0,0,0,...,2,1,0,9,0,2,0,15,22,1
3,3,1,1,0,1,0,1,1,0,0,...,2,2,2,5,1,0,0,0,0,1
4,4,2,1,0,1,0,1,1,0,1,...,0,1,0,9,0,1,0,19,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2,0,0,1,0,0,0,1,0,...,2,1,2,9,2,0,0,0,0,1
26703,26703,1,2,0,1,0,1,0,0,0,...,2,2,0,6,1,1,0,5,3,1
26704,26704,2,2,0,1,1,1,1,0,1,...,2,1,1,6,0,0,0,0,0,1
26705,26705,1,1,0,0,0,0,0,0,2,...,0,2,0,5,2,1,0,5,7,1


In [15]:
test

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,target
0,26707,2,2,0,1,0,1,1,0,1,...,2,2,0,7,0,1,0,2,8,-1
1,26708,1,1,0,0,0,0,0,0,0,...,2,2,0,1,2,3,0,2,21,-1
2,26709,2,2,0,0,1,1,1,1,1,...,0,1,0,5,2,1,0,11,13,-1
3,26710,1,1,0,0,0,0,0,0,0,...,0,1,2,5,0,1,0,0,0,-1
4,26711,3,1,1,1,0,1,1,1,1,...,2,1,0,6,2,0,1,5,11,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26703,53410,1,1,0,1,0,1,0,0,1,...,1,0,1,2,1,1,1,0,0,-1
26704,53411,3,1,0,1,0,1,1,1,1,...,0,2,0,9,2,1,3,5,19,-1
26705,53412,0,1,0,0,0,0,0,0,0,...,2,2,2,9,0,1,0,0,0,-1
26706,53413,3,1,0,1,0,1,0,1,0,...,0,1,2,1,0,1,0,0,0,-1


In [16]:
# Will drop columns like target and respondent id
X.drop(['target'], axis=1, inplace=True)
test.drop(['target'], axis=1, inplace=True)
X.drop(['respondent_id'], axis=1, inplace=True)
test.drop(['respondent_id'], axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [17]:
X

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,1,0,0,0,0,0,0,1,1,0,...,2,2,1,2,8,2,0,0,0,0
1,3,2,0,1,0,1,0,1,1,0,...,2,2,2,0,1,0,0,0,13,20
2,1,1,0,1,0,0,0,0,0,2,...,0,2,1,0,9,0,2,0,15,22
3,1,1,0,1,0,1,1,0,0,0,...,2,2,2,2,5,1,0,0,0,0
4,2,1,0,1,0,1,1,0,1,0,...,0,0,1,0,9,0,1,0,19,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,2,0,0,1,0,0,0,1,0,0,...,0,2,1,2,9,2,0,0,0,0
26703,1,2,0,1,0,1,0,0,0,1,...,0,2,2,0,6,1,1,0,5,3
26704,2,2,0,1,1,1,1,0,1,0,...,3,2,1,1,6,0,0,0,0,0
26705,1,1,0,0,0,0,0,0,2,0,...,0,0,2,0,5,2,1,0,5,7


In [18]:
test

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,2,2,0,1,0,1,1,0,1,0,...,1,2,2,0,7,0,1,0,2,8
1,1,1,0,0,0,0,0,0,0,0,...,2,2,2,0,1,2,3,0,2,21
2,2,2,0,0,1,1,1,1,1,0,...,1,0,1,0,5,2,1,0,11,13
3,1,1,0,0,0,0,0,0,0,1,...,0,0,1,2,5,0,1,0,0,0
4,3,1,1,1,0,1,1,1,1,0,...,0,2,1,0,6,2,0,1,5,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26703,1,1,0,1,0,1,0,0,1,1,...,3,1,0,1,2,1,1,1,0,0
26704,3,1,0,1,0,1,1,1,1,0,...,2,0,2,0,9,2,1,3,5,19
26705,0,1,0,0,0,0,0,0,0,0,...,2,2,2,2,9,0,1,0,0,0
26706,3,1,0,1,0,1,0,1,0,0,...,0,0,1,2,1,0,1,0,0,0


In [19]:
Y_1 =pd.read_csv("../input/flu-shot-prediction/training_set_labels.csv")

In [20]:
# We can see that there are 3 columns in training set labels which is our target data
Y_1.drop(['respondent_id'], axis=1, inplace=True)
Y_1.drop(['seasonal_vaccine'],axis=1,inplace=True)

In [21]:
# I have dropped 2 columns for now our target is h1n1_vaccine
Y_1

,h1n1_vaccine
0,0
1,0
2,0
3,0
4,0
...,...
26702,0
26703,0
26704,0
26705,0


In [22]:
# import re
# regex = re.compile(r"\[|\]|<", re.IGNORECASE)
# X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]

In [23]:
# test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in test.columns.values]

In [24]:
discrete_features = X.dtypes == int

In [25]:
# Im figuring which feature is contributing the most using mi_scores
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, Y_1, discrete_features)
mi_scores

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


doctor_recc_h1n1               0.071119
opinion_h1n1_risk              0.052467
opinion_h1n1_vacc_effective    0.049233
opinion_seas_risk              0.028989
doctor_recc_seasonal           0.028919
health_insurance               0.028244
employment_industry            0.017145
employment_occupation          0.016145
opinion_seas_vacc_effective    0.015682
h1n1_concern                   0.011315
health_worker                  0.010173
h1n1_knowledge                 0.008210
chronic_med_condition          0.004864
sex                            0.003818
census_msa                     0.003746
behavioral_face_mask           0.003557
race                           0.003495
opinion_seas_sick_from_vacc    0.003395
behavioral_touch_face          0.003322
behavioral_wash_hands          0.002706
behavioral_antiviral_meds      0.001436
behavioral_avoidance           0.001341
income_poverty                 0.001095
rent_or_own                    0.000665
behavioral_large_gatherings    0.000309


In [26]:
# The mi_score of these features is almost equal to zero so I'm removing them
remove_features=['education','sex','employment_status','hhs_geo_region','census_msa','household_children']

In [27]:
X.drop(remove_features, axis=1, inplace=True)
test.drop(remove_features, axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [28]:
X

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,race,income_poverty,marital_status,rent_or_own,household_adults,employment_industry,employment_occupation
0,1,0,0,0,0,0,0,1,1,0,...,0,1,3,3,2,2,1,0,0,0
1,3,2,0,1,0,1,0,1,1,0,...,1,3,1,3,2,2,2,0,13,20
2,1,1,0,1,0,0,0,0,0,2,...,0,1,0,3,0,2,1,2,15,22
3,1,1,0,1,0,1,1,0,0,0,...,3,0,4,3,2,2,2,0,0,0
4,2,1,0,1,0,1,1,0,1,0,...,0,3,2,3,0,0,1,1,19,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,2,0,0,1,0,0,0,1,0,0,...,1,1,4,3,0,2,1,0,0,0
26703,1,2,0,1,0,1,0,0,0,1,...,0,0,0,3,0,2,2,1,5,3
26704,2,2,0,1,1,1,1,0,1,0,...,3,1,3,3,3,2,1,0,0,0
26705,1,1,0,0,0,0,0,0,2,0,...,0,1,0,1,0,0,2,1,5,7


In [29]:
# Splitting the training data into training and validation data
from sklearn.model_selection import train_test_split
X_train_1, X_valid_1, y_train, y_valid = train_test_split(X, Y_1, train_size=0.90, test_size=0.10,
                                                                random_state=0)


In [30]:
X_train_1, X_valid_1 = X_train_1.align(X_valid_1, join='left', axis=1)
X_train_1, test = X_train_1.align(test, join='left', axis=1)

In [31]:
#We'll choose XGboost regressor as our model
from xgboost import XGBRegressor

# Define the model
my_model_1 = XGBRegressor(learning_rate =0.05,
                           n_estimators=2000) # Your code here

# Fit the model
my_model_1.fit(X_train_1,y_train,
             early_stopping_rounds=10, 
             eval_set=[(X_valid_1, y_valid)],
             verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=2000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [32]:
# auc_prediction=my_model_1.predict(X_valid_1)

In [33]:
# from sklearn.metrics import roc_auc_score
# score=roc_auc_score(auc_prediction,y_valid)

In [34]:
predictions_1 = my_model_1.predict(test)

In [35]:
# predictions_1=(predictions_1).astype(float)

In [36]:
predictions_1.size

26708

We have got the predictions of our first target on test data

In [37]:
#Now our target is seasonal_vaccine
Y_2 =pd.read_csv("../input/flu-shot-prediction/training_set_labels.csv")
Y_2.drop(['respondent_id'], axis=1, inplace=True)
Y_2.drop(['h1n1_vaccine'],axis=1,inplace=True)

In [38]:
Y_2

,seasonal_vaccine
0,0
1,1
2,0
3,1
4,0
...,...
26702,0
26703,0
26704,1
26705,0


In [39]:
#Again splitting the training data into training and validation data
from sklearn.model_selection import train_test_split
X_train_2, X_valid_2, y_train_2, y_valid_2 = train_test_split(X, Y_2, train_size=0.9, test_size=0.1,
                                                                random_state=0)


In [40]:
X_train_2, X_valid_2 = X_train_2.align(X_valid_2, join='left', axis=1)
X_train_2, test = X_train_2.align(test, join='left', axis=1)

In [41]:
# We'll use xg-boost regressor for this as well
# Define the model
my_model_2 = XGBRegressor(learning_rate =0.05,
                           n_estimators=2000)# Your code here

# Fit the model
my_model_2.fit(X_train_2,y_train_2,
             early_stopping_rounds=10, 
             eval_set=[(X_valid_2, y_valid_2)],
             verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=2000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [42]:
predictions_2 = my_model_2.predict(test)

In [43]:
# predictions_2=(predictions_2).astype(float)

In [44]:
predictions_2.size

26708

We have got the predictions of our second target on test data as well

In [45]:
id=pd.read_csv("../input/flu-shot-prediction/test_set_features.csv")

In [46]:
# Creating a dataframe containing respondent_id and the predictions of both the targets
output = pd.DataFrame({'respondent_id': id.respondent_id, 'h1n1_vaccine': predictions_1, 'seasonal_vaccine':predictions_2})

In [47]:
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
